In [1]:
# look for missing files throughout pipeline

# import modules
from glob import glob
import re
import os.path
from itertools import repeat
import csv
from shutil import copyfile
import shutil
import pandas as pd

In [2]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [52]:
root = '/om/group/saxelab/OpenAutism/data/' #+site+'/'+subject_name+'/dicom'
other_folders = ['mriqc_output', 'BIDS', '_OLD_', 'SPM_firstlevel']

all_sites_dir = glob(root+'*/')
all_sites = []
all_subjects = []

missing_dicoms = []
missing_anat = []
missing_func = []
missing_bids = []
missing_fmriprep = []
missing_generaltsv = []
missing_mriqc_folder = []
missing_mriqc_T1 = []
missing_fmriprep_smoothpreproc = []
missing_fmriprep_confounds = []
missing_mriqc_func = []
missing_firstlevel = []
missing_firstleveltime = []

for sites in all_sites_dir:
    sites = re.split("/+", sites)
    sites = sites[-2]
    all_sites.append(sites)

all_sites = [ N for N in all_sites if "TextFiles" not in N ]
all_sites = [ N for N in all_sites if "subjectlist" not in N ]

# all_sites is a list containing names of sites

for sites in all_sites:
    all_subjects_dir = glob(root+sites+"/"+'*/')
    
    for element in all_subjects_dir:                # all_subjects_dir is the long directory (one site)
            subjects = re.split("/+", element)
            subjects = subjects[-2]                 # subjects is the SUB_SITE_NNN format
            subjects_str = subjects.replace("_","") # subjects_str is the SUBSITENNN format
            if subjects not in str(other_folders):
                all_subjects.append(subjects)      # saving this for later (total amount of subjects based on folder names)
        
                # is dicom folder present? (permissions do not allow me to see if non-empty)
                dicom_dir = os.listdir(root+sites+'/'+subjects+'/')
                if 'dicom' not in dicom_dir:
                    missing_dicoms.append(sites+subjects)
                else:
                    missing_dicoms.append("")
                    
                # BIDS RELATED::
                    
                # is BIDS subject folder present?
                if not (os.path.isdir(root+sites+'/BIDS/sub-'+subjects_str+'/')):
                    missing_bids.append(sites+subjects)   
                    missing_generaltsv.append(sites+subjects)  
                    missing_anat.append(sites+subjects)
                    missing_func.append(sites+subjects)  
                else:           # if BIDS folder exists search for files:
                    missing_bids.append("")
                    #missing_generaltsv.append("")  
                    
                    # is BIDS .tsv file present?
                    if not (os.path.isfile(root+sites+'/BIDS/sub-'+subjects_str+'/sub-'+subjects_str+"_scans.tsv")):
                        missing_generaltsv.append(sites+subjects)   
                    else:
                        missing_generaltsv.append("")  

                    # is BIDS ANAT files present?
                    if not (os.path.isfile(root+sites+'/BIDS/sub-'+subjects_str+'/anat/'+'sub-'+subjects_str+'_T1w.json') and os.path.isfile(root+sites+'/BIDS/sub-'+subjects_str+'/anat/'+'sub-'+subjects_str+'_T1w.nii.gz')):
                        missing_anat.append(sites+subjects)
                    else:
                        missing_anat.append("")
                    
                    # is BIDS FUNC folder present?
                    all_func_nifti = []
                    all_func_json = []
                    all_func_events = []
                    if os.path.isdir(root+sites+'/BIDS/sub-'+subjects_str+'/func/'):
                        func_files = os.listdir(root+sites+'/BIDS/sub-'+subjects_str+'/func/')
                        missing_func.append("") 
                        for funcs in func_files:
                            # make sure same amount of json, nii.gz and tsv per run
                            all_func_json.append(find_between(funcs, "task-", "_bold.json" ))
                            all_func_nifti.append(find_between(funcs, "task-", "_bold.nii.gz" ))
                            all_func_events.append(find_between(funcs, "task-", "_events.tsv" ))
                        # improve this by making it by name
                        if not (len(all_func_json) == len(all_func_nifti) == len(all_func_events)):
                            print("Missing files in BIDS>>func folder for subject "+subjects_str)
                    else:
                        missing_func.append(sites+subjects) 
                            
                
                ############################
                
                # list of runs by niftis in bids
                all_func_nifti = [x for x in all_func_nifti if x]
                len_funcs = len(all_func_nifti)
                
                # is fmriprep folder present?
                missing_fmriprep_funcs = 0
                missing_fmriprep_confs = 0
                try:  
                    listdir = os.listdir(root+sites+'/BIDS/derivatives/fmriprep/sub-'+subjects_str+'/func/')
                    missing_fmriprep.append("")
                    for funcs in all_func_nifti:
                        preproc = "sub-"+subjects_str+"_task-"+funcs+"_bold_space-MNI152NLin2009cAsym_variant-smoothAROMAnonaggr_preproc.nii.gz"
                        confounds = "sub-"+subjects_str+"_task-"+funcs+"_bold_confounds.tsv"
                        # add mriqc
                        
                        if preproc not in listdir:
                            missing_fmriprep_funcs = 1
                        if confounds not in listdir:
                            missing_fmriprep_confs = 1
                except FileNotFoundError:
                    missing_fmriprep.append(sites+subjects)
                    missing_fmriprep_funcs = 1
                    missing_fmriprep_confs = 1
                    
                if missing_fmriprep_funcs == 0:
                    missing_fmriprep_smoothpreproc.append("")
                else:
                    missing_fmriprep_smoothpreproc.append(sites+subjects)
                    
                if missing_fmriprep_confs == 0:
                    missing_fmriprep_confounds.append("")
                else:
                    missing_fmriprep_confounds.append(sites+subjects)          
                
                # is mriqc complete?
                #filename_re = "sub-"subjects_str #mriqc report with subject name
                quality = 0
                try: 
                    tempdir = os.listdir(root+sites+'/mriqc_output/reports/')
                    missing_mriqc_folder.append("")
                    if r"sub-"+subjects_str+"_T1w.html" in tempdir:
                        missing_mriqc_T1.append("")
                    else:
                        missing_mriqc_T1.append(sites+subjects)
                    for funcs in all_func_nifti:
                        QI = "sub-"+subjects_str+"_task-"+funcs+"_bold.html"
                        if QI not in tempdir:
                            quality = 1
                except FileNotFoundError:
                    missing_mriqc_T1.append(sites+subjects)
                    missing_mriqc_folder.append(sites+subjects)
                    quality = 1
                    
                if quality == 0:
                    missing_mriqc_func.append("")
                else:
                    missing_mriqc_func.append(sites+subjects) 
                
                # now look for missing first level anayses
                first_lev_error = 0
                first_level_timeseries_error = 0
                firstleveldir = '/om/group/saxelab/OpenAutism/Analysis/first_level_standard.py/'+ sites+'/sub-'+subjects_str
                firstleveltimedir = '/om/group/saxelab/OpenAutism/Analysis/first_level_standard_timeseries.py/'+ sites+'/sub-'+subjects_str
                sites_low = sites.lower()
                
                if sites == 'TASTI2':
                    subjects_str = sites_low
                
                if os.path.isdir(firstleveldir) or os.path.isdir(firstleveltimedir):
                    for funcs in all_func_nifti:
                        cut_task = funcs.split('_', 1)[0]
                        cut_run = "run"+(funcs.split('_', 1)[-1][-1])
                        firstleveldir = '/om/group/saxelab/OpenAutism/Analysis/first_level_aroma.py/'+sites+'/sub-'+subjects_str+'/'+cut_task+'/model/'+cut_run
                        firstleveltimedir = '/om/group/saxelab/OpenAutism/Analysis/first_level_standard_timeseries.py/'+ sites+'/sub-'+subjects_str+'/'+cut_task+'/model/'+cut_run
                        try:
                            if len(os.listdir(firstleveldir) ) == 0:
                                first_lev_error = 1
                                first_level_timeseries_error = 1
                        except FileNotFoundError:
                            first_lev_error = 1
                            first_level_timeseries_error = 1
                else:
                    if os.path.isdir(firstleveldir) == 0:
                        first_lev_error = 1
                    if os.path.isdir(firstleveldir) == 0: # in addition
                        first_level_timeseries_error = 1
                    
                    
                if first_lev_error == 0:
                    missing_firstlevel.append("")
                else:
                    missing_firstlevel.append(sites+subjects) 
                    
                if first_level_timeseries_error == 0:
                    missing_firstleveltime.append("")
                else:
                    missing_firstleveltime.append(sites+subjects) 
                   

d = {'all_subjects': all_subjects, 'missing_dicoms': missing_dicoms, 'missing_bids': missing_bids,
    'missing_generaltsv': missing_generaltsv, 'missing_anat':missing_anat, 'missing_func':missing_anat,
    'missing_fmriprep':missing_fmriprep, 'missing_fmriprep_smoothpreproc':missing_fmriprep_smoothpreproc,
     'missing_fmriprep_confounds':missing_fmriprep_confounds,'missing_mriqc_T1':missing_mriqc_T1,
    'missing_mriqc_func':missing_mriqc_func, 'missing_firstlevel':missing_firstlevel, 'missing_firstleveltime': missing_firstleveltime}
    
df = pd.DataFrame(data=d, columns=['all_subjects', 'missing_dicoms', 'missing_bids','missing_generaltsv', 'missing_anat', 'missing_func', 
                                  'missing_fmriprep', 'missing_fmriprep_smoothpreproc', 'missing_fmriprep_confounds','missing_mriqc_T1', 'missing_mriqc_func', 'missing_firstlevel','missing_firstleveltime'])

df.to_csv('/om/user/rezzo/list_of_missing_files')


In [53]:
df

,all_subjects,missing_dicoms,missing_bids,missing_generaltsv,missing_anat,missing_func,missing_fmriprep,missing_fmriprep_smoothpreproc,missing_fmriprep_confounds,missing_mriqc_T1,missing_mriqc_func,missing_firstlevel,missing_firstleveltime
0,SAX_DOD_010,,,,,,,,,,,DODSAX_DOD_010,DODSAX_DOD_010
1,SAX_DOD_016,,,,,,,,,,,DODSAX_DOD_016,DODSAX_DOD_016
2,SAX_DOD_027,,,,,,,,,DODSAX_DOD_027,DODSAX_DOD_027,,
3,SAX_DOD_003,,,,,,,,,DODSAX_DOD_003,DODSAX_DOD_003,,
4,SAX_DOD_041,,,,,,,,,DODSAX_DOD_041,DODSAX_DOD_041,,
5,SAX_DOD_015,,,,,,,,,,,,
6,SAX_DOD_008,,,,,,,,,,,,
7,SAX_DOD_030,,,,,,,,,DODSAX_DOD_030,DODSAX_DOD_030,DODSAX_DOD_030,DODSAX_DOD_030
8,SAX_DOD_002,,,,,,,,,DODSAX_DOD_002,DODSAX_DOD_002,DODSAX_DOD_002,DODSAX_DOD_002
9,SAX_DOD_053,,,,,,,,,,,DODSAX_DOD_053,DODSAX_DOD_053
